<a href="https://colab.research.google.com/github/yahui624/CompArchFinalProject/blob/main/cudaSetup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test CUDA Google Collab enviorment setup following [cuda-lab
](https://github.com/alessandrobessi/cuda-lab)

In [7]:
!git clone https://github.com/alessandrobessi/cuda-lab.git
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

fatal: destination path 'cuda-lab' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-bif2f5cr
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-bif2f5cr
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=5b6010eb525187410665b7649f3eba14347c247b0354b6ac6a8ba83b74a88bba
  Stored in directory: /tmp/pip-ephem-wheel-cache-q648g1b0/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [5]:
!nvidia-smi
!nvcc --version
!nvprof --version

Mon Dec 12 09:53:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [12]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [13]:
!nvcc cuda-lab/add.cu -o add -Wno-deprecated-gpu-targets

In [14]:
!nvprof ./add

==341== NVPROF is profiling process 341, command: ./add
Max error: 0.000000
==341== Profiling application: ./add
==341== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.5406ms         1  2.5406ms  2.5406ms  2.5406ms  add(int, float*, float*)
      API calls:   98.32%  278.17ms         2  139.08ms  61.182us  278.11ms  cudaMallocManaged
                    0.91%  2.5628ms         1  2.5628ms  2.5628ms  2.5628ms  cudaDeviceSynchronize
                    0.56%  1.5736ms         2  786.78us  321.01us  1.2525ms  cudaFree
                    0.13%  355.90us         1  355.90us  355.90us  355.90us  cuDeviceTotalMem
                    0.06%  156.50us       101  1.5490us     132ns  66.628us  cuDeviceGetAttribute
                    0.02%  57.624us         1  57.624us  57.624us  57.624us  cudaLaunchKernel
                    0.01%  32.379us         1  32.379us  32.379us  32.379us  cuDeviceGetName
                  

In [15]:
!time ./add

Max error: 0.000000

real	0m0.825s
user	0m0.029s
sys	0m0.732s


In [16]:
!nsys profile -t nvtx,cuda --stats=true --force-overwrite true --wait=all -o my_report ./add


Max error: 0.000000

The target application terminated with signal 11 (SIGSEGV)
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-7fe7-8320-d214-7c30.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-7fe7-8320-d214-7c30.qdrep"
Exporting 2967 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-7fe7-8320-d214-7c30.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls     Average      Minimum     Maximum            Name         
 -------  ---------------  ---------  -------------  ---------  -----------  ---------------------
    98.7      237,456,709          2  118,728,354.5     55,765  237,400,944  cudaMallocManaged    
     1.0        2,424,870          1    2,424,870.0  2,424,870    2,424,870  cudaDeviceSynchronize
     0.3          617,506          2      308,

In [9]:
%%cu
#include <iostream>
	int
	main()
{
	std::cout << "Welcome To GeeksforGeeks\n";
	return 0;
}


Welcome To GeeksforGeeks



In [10]:
%%cu
#include <cstdio>
#include <iostream>

	using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


The maximum element is : 772171792
The time required : 0.003008

